In [1]:
import os
import sys
import pandas as pd
import numpy as np
from numpy import diff
import matplotlib.pyplot as plt
import re

%matplotlib inline
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
pd.options.mode.chained_assignment = None #отключить варнинги на изменение оригинального датафрейма

In [2]:
source_dir = 'C:/Users/Barca/Desktop/GitHub/IPO_Analysis/cb_2022/'

In [3]:
rounds = pd.read_csv(source_dir + 'solved_rounds.csv')
rounds.head(3)

,Unnamed: 0,Organization Name,Announced Date,Money Raised,Pre-Money Valuation,Total Funding Amount,Lead Investors,Funding Type,Organization Industries,Number of Funding Rounds,Number of Investors,Number of Partner Investors,Org_Code,Round_Code,exit_date,exit_type,Valuation,Raised Money,Funding Amount
0,1,&pizza,2014-09-01,1900000.0,NaN,79200000.0,NaN,Seed,"Food and Beverage,Food Delivery,Food Processing,Snack Food",9,NaN,NaN,/organization/pizza-4,/funding_round/pizza-4-seed--5f65c774,NaN,NaN,NaN,NaN,NaN
1,2,&pizza,2014-12-01,10000000.0,NaN,79200000.0,NaN,Funding Round,"Food and Beverage,Food Delivery,Food Processing,Snack Food",9,1.0,NaN,/organization/pizza-4,/funding_round/pizza-4-undisclosed--d9bc03f1,NaN,NaN,NaN,NaN,NaN
2,3,&pizza,2015-02-18,10000000.0,NaN,79200000.0,NaN,Venture - Series Unknown,"Food and Beverage,Food Delivery,Food Processing,Snack Food",9,NaN,NaN,/organization/pizza-4,/funding_round/pizza-4-series-unknown--3bdd84b3,NaN,NaN,NaN,NaN,NaN


### Категории

In [4]:
category_list = rounds[['Organization Industries']]
category_list = category_list.loc[~category_list['Organization Industries'].isin([np.nan])]
category_list.groupby(['Organization Industries']).size().reset_index(name='count')

,Organization Industries,count
0,"3D Printing,3D Technology",2
1,"3D Printing,3D Technology,Artificial Intelligence,Biotechnology,Medical Device",3
2,"3D Printing,3D Technology,CAD,Mechanical Engineering,Software",2
3,"3D Printing,3D Technology,Consumer Electronics,Hardware,Manufacturing",6
4,"3D Printing,3D Technology,Consumer Electronics,Robotics,Software",3
...,...,...
14522,"iOS,Mobile,Real Time,Software",3
14523,"iOS,Mobile,Software",3
14524,"iOS,Software",3
14525,"iOS,macOS,Mobile Devices,Software",2


### Теперь с запятыми

In [5]:
# category_list = rounds[['Organization Industries']]
# category_list = category_list.loc[~category_list['Organization Industries'].isin([np.nan])]
# words = {}
# for i in range(len(category_list)):
#     string = category_list.iloc[i, :]['Organization Industries'].split(',')
#     for word in string:
#         if word not in words:
#             words[word] = 1
#         else:
#             words[word] += 1
        
# category_list = pd.DataFrame.from_dict(words, orient='index')
# category_list.to_csv('category_list.csv', index=True, encoding = 'utf-8-sig')
# category_list

### Таблица попарных встечаний тегов

In [6]:
categories = pd.read_csv('Industry categories 2.csv')
#categories = categories.drop(['ID'], axis=1)
categories = categories.loc[categories['Category'] != '-']
categories['Name'] = categories['Name'].str.lstrip()
categories

,ID,Name,Category
0,1,Software,Software
1,2,Health Care,Health
2,3,Biotechnology,Biotech
3,4,Information Technology,Software
4,5,Enterprise Software,Software
...,...,...,...
703,704,Sailing,EComm
704,705,SNS,Software
705,706,Social Assistance,Health
706,707,Sponsorship,AdMedia


In [7]:
categories.groupby(['Category']).size().reset_index(name = 'count')

,Category,count
0,AdMedia,118
1,Biotech,10
2,EComm,162
3,FinTech,48
4,Health,36
5,Infra,37
6,Physical,90
7,Software,206


1 = Software (112735  206)  
2 = EComm (30089  162)  
3 = AdMedia (21046  118)  
4 = Physical (23657  90)  
5 = FinTech (18129  48)  
6 = Infra (15860  37)  
7 = Health (32407  36)  
8 = Biotech (20811  10)  

In [8]:
categories.loc[categories['Category'] == 'Software', 'tag'] = 1
categories.loc[categories['Category'] == 'EComm', 'tag'] = 2
categories.loc[categories['Category'] == 'AdMedia', 'tag'] = 3
categories.loc[categories['Category'] == 'Physical', 'tag'] = 4
categories.loc[categories['Category'] == 'FinTech', 'tag'] = 5
categories.loc[categories['Category'] == 'Infra', 'tag'] = 6
categories.loc[categories['Category'] == 'Health', 'tag'] = 7
categories.loc[categories['Category'] == 'Biotech', 'tag'] = 8
categories['tag'] = categories['tag'].astype(int)

aaa = dict(zip(categories['Name'],categories['tag']))
aaa

{'Software': 1,
 'Health Care': 7,
 'Biotechnology': 8,
 'Information Technology': 1,
 'Enterprise Software': 1,
 'SaaS': 1,
 'Medical': 7,
 'Internet': 1,
 'Analytics': 1,
 'Manufacturing': 4,
 'Artificial Intelligence': 1,
 'E-Commerce': 2,
 'Mobile': 1,
 'Medical Device': 7,
 'Financial Services': 5,
 'FinTech': 5,
 'Therapeutics': 8,
 'Pharmaceutical': 8,
 'Machine Learning': 1,
 'Advertising': 3,
 'Big Data': 1,
 'Security': 1,
 'Health Diagnostics': 7,
 'Cloud Computing': 1,
 'Finance': 5,
 'Energy': 4,
 'Telecommunications': 6,
 'Cyber Security': 1,
 'Apps': 1,
 'Hardware': 6,
 'Wireless': 6,
 'Marketing': 3,
 'Electronics': 6,
 'Life Science': 8,
 'Retail': 2,
 'Wellness': 7,
 'Computer': 1,
 'Video': 3,
 'Semiconductor': 6,
 'Food and Beverage': 2,
 'Information Services': 1,
 'Education': 2,
 'Marketplace': 2,
 'Mobile Apps': 1,
 'Biopharma': 8,
 'Network Security': 1,
 'Social Media': 3,
 'Real Estate': 2,
 'Renewable Energy': 4,
 'Robotics': 4,
 'Payments': 5,
 'Digital Med

In [9]:
tags = rounds[['Organization Name', 'Org_Code', 'Round_Code', 'Organization Industries']]
tags = tags.loc[~tags['Organization Industries'].isin([np.nan])]
tags.loc[tags['Organization Industries'].isin([np.nan]), 'Organization Industries'] = ''
tags['tag'] = tags['Organization Industries']
tags['tag'] = tags['tag'].replace({'^,':''}, regex=True)
tags['tag'] = tags['tag'].replace({',$':''}, regex=True)
tags = tags[['Organization Name', 'Org_Code', 'Round_Code', 'tag']].reset_index(drop = True)
tags['tag'] = tags['tag'].str.split(',').map(set).str.join(',')
tags['final'] = 0
tags['if_same_count'] = 0
tags

,Organization Name,Org_Code,Round_Code,tag,final,if_same_count
0,&pizza,/organization/pizza-4,/funding_round/pizza-4-seed--5f65c774,"Food Processing,Food and Beverage,Food Delivery,Snack Food",0,0
1,&pizza,/organization/pizza-4,/funding_round/pizza-4-undisclosed--d9bc03f1,"Food Processing,Food and Beverage,Food Delivery,Snack Food",0,0
2,&pizza,/organization/pizza-4,/funding_round/pizza-4-series-unknown--3bdd84b3,"Food Processing,Food and Beverage,Food Delivery,Snack Food",0,0
3,&pizza,/organization/pizza-4,/funding_round/pizza-4-series-unknown--819c151c,"Food Processing,Food and Beverage,Food Delivery,Snack Food",0,0
4,&pizza,/organization/pizza-4,/funding_round/pizza-4-convertible-note--7b31885f,"Food Processing,Food and Beverage,Food Delivery,Snack Food",0,0
...,...,...,...,...,...,...
67594,zulily,/organization/zulily,/funding_round/zulily-series-a--231238fa,"Retail,E-Commerce,Delivery",0,0
67595,zulily,/organization/zulily,/funding_round/zulily-series-b--c1175b56,"Retail,E-Commerce,Delivery",0,0
67596,zulily,/organization/zulily,/funding_round/zulily-series-c--80316213,"Retail,E-Commerce,Delivery",0,0
67597,zulily,/organization/zulily,/funding_round/zulily-series-d--fe4ecba4,"Retail,E-Commerce,Delivery",0,0


In [10]:
# count_5 = 0
# count_4 = 0
# for i in range(len(tags)):
#     string = tags.iloc[i, :]['tag'].split(',')
#     original = string.copy()
#     for k in range(len(string)):
#         string[k] = aaa[string[k]]
#     for tag in string:
#         if tag == 5:
#             count_5 += 1
#             if 3 in string:
#                 count_4 +=1
# print(count_5)
# print(count_4)

In [11]:
# tab = pd.DataFrame(columns=[1, 2, 3, 4, 5, 6, 7, 8], index=[0, 1, 2, 3, 4, 5, 6, 7, 8])
# tab = tab.fillna(value=0)

# for i in range(len(tags)):
#     string = tags.iloc[i, :]['tag'].split(',')
#     #print(string)
#     for i in range(len(string)):
#         string[i] = aaa[string[i]]
#     #print(string)
#     d = {1:0, 2:0, 3:0, 4:0, 5:0, 6:0, 7:0, 8:0}
#     for tag in string:
#         if tag in d:
#             d[tag] += 1
#     #print(d)
#     single = 0
#     tatata = []
#     for tag in d:
#         if d[tag] != 0:
#             single += 1
#             tatata.append(tag)
#     #print(tatata)
#     #print(single)
#     if single == 1:
#         tab.loc[0, tatata[0]] += 1
#     else:
#         for tag in tatata:
#             if d[tag] >= 2:
#                 tab.loc[tag, tag] += 1
#             for tag2 in tatata:
#                 if tag2 != tag:
#                     tab.loc[tag, tag2] += 1

# tab.to_csv('taxonomia.csv', index=True, encoding = 'utf-8-sig')
# tab

In [12]:
tags.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 67599 entries, 0 to 67598
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Organization Name  67599 non-null  object
 1   Org_Code           67599 non-null  object
 2   Round_Code         67599 non-null  object
 3   tag                67599 non-null  object
 4   final              67599 non-null  int64 
 5   if_same_count      67599 non-null  int64 
dtypes: int64(2), object(4)
memory usage: 3.1+ MB


In [12]:
for i in range(len(tags)):
    string = tags.iloc[i, :]['tag'].split(',')
    #print(string)
    for k in range(len(string)):
        string[k] = aaa[string[k]]
    #print(string)
    d = {8:0, 7:0, 6:0, 5:0, 4:0, 3:0, 2:0, 1:0}
    for tag in string:
        if tag in d:
            d[tag] += 1
    #print(d)
    max_tag = max(d, key=d.get)
    max_count = max(d[tag] for tag in d)
    tags.at[i, 'final'] = max_tag
    #print(max_tag)
    #print(max_count)
    for tag in [2,3,4,5,6,7,8]:
        if (tag != max_tag) and (d[tag] == max_count):
            tags.at[i, 'if_same_count'] = tag
            #print(tag)
            
tags.to_csv('tags.csv', index=False, encoding = 'utf-8-sig')
tags

,Organization Name,Org_Code,Round_Code,tag,final,if_same_count
0,&pizza,/organization/pizza-4,/funding_round/pizza-4-seed--5f65c774,"Food and Beverage,Food Processing,Food Delivery,Snack Food",2,0
1,&pizza,/organization/pizza-4,/funding_round/pizza-4-undisclosed--d9bc03f1,"Food and Beverage,Food Processing,Food Delivery,Snack Food",2,0
2,&pizza,/organization/pizza-4,/funding_round/pizza-4-series-unknown--3bdd84b3,"Food and Beverage,Food Processing,Food Delivery,Snack Food",2,0
3,&pizza,/organization/pizza-4,/funding_round/pizza-4-series-unknown--819c151c,"Food and Beverage,Food Processing,Food Delivery,Snack Food",2,0
4,&pizza,/organization/pizza-4,/funding_round/pizza-4-convertible-note--7b31885f,"Food and Beverage,Food Processing,Food Delivery,Snack Food",2,0
...,...,...,...,...,...,...
67594,zulily,/organization/zulily,/funding_round/zulily-series-a--231238fa,"Delivery,E-Commerce,Retail",2,0
67595,zulily,/organization/zulily,/funding_round/zulily-series-b--c1175b56,"Delivery,E-Commerce,Retail",2,0
67596,zulily,/organization/zulily,/funding_round/zulily-series-c--80316213,"Delivery,E-Commerce,Retail",2,0
67597,zulily,/organization/zulily,/funding_round/zulily-series-d--fe4ecba4,"Delivery,E-Commerce,Retail",2,0


In [13]:
tags = tags.drop(['Round_Code'], axis=1)
tags = tags.drop_duplicates()
tags

,Organization Name,Org_Code,tag,final,if_same_count
0,&pizza,/organization/pizza-4,"Food and Beverage,Food Processing,Food Delivery,Snack Food",2,0
5,(mt) Media Temple,/organization/media-temple,"Cloud Management,Web Hosting,Cloud Computing",1,0
8,0x,/organization/0x-project,"Blockchain,Ethereum,Cryptocurrency",5,0
10,10 Federal,/organization/10-federal,"Self-Storage,Rental,Leasing",2,0
13,10-4 Systems,/organization/10-4-systems,"Mobile,Transportation,Software",1,0
...,...,...,...,...,...
67582,yello,/organization/yello,"Mobile,Recruiting,Software,SaaS,CRM",1,0
67587,you.com,/organization/you-com-523e,"Search Engine,Internet,Information Technology",1,0
67589,zSpace,/organization/zspace,"Education,STEM Education,Social Impact,Software,Innovation Management,Virtual Reality",1,0
67592,zecOps,/organization/zecops-inc,"Computer,Network Security,Information Technology",1,0


In [14]:
tags.loc[tags['if_same_count'] != 0]

,Organization Name,Org_Code,tag,final,if_same_count
30,10X Genomics,/organization/10x-genomics,"Genetics,Biotechnology,Health Care,Medical",8,7
96,20/20 Gene Systems,/organization/20-20-gene-systems-inc,"Biotechnology,Health Diagnostics,Big Data",8,7
151,3-Dimensional Pharmaceuticals,/organization/3-dimensional-pharmaceuticals,"Pharmaceutical,Health Care",8,7
164,360fly,/organization/360fly-inc,"Video,Consumer Electronics,Software",6,3
170,3BG Supply Co.,/organization/3bg-supply-co,"Service Industry,Industrial,Information Technology",4,2
...,...,...,...,...,...
67252,nanoMR,/organization/nanomr,"Developer Platform,Biotechnology,Health Care",8,7
67258,neoSaej,/organization/neosaej,"Auctions,E-Commerce,Internet,Insurance,Financial Services,Marketplace,Banking",5,2
67349,quip,/organization/quip-2,"Product Design,E-Commerce,Dental,Beauty,Health Care",7,2
67370,reThought Insurance,/organization/rethought-insurance,"Innovation Management,InsurTech,Commercial",5,2


In [15]:
tags.groupby(['final']).size().reset_index(name = 'count')

,final,count
0,1,6871
1,2,1807
2,3,1321
3,4,1573
4,5,1415
5,6,1211
6,7,2480
7,8,2079


1 = Software (112735  206)  
2 = EComm (30089  162)  
3 = AdMedia (21046  118)  
4 = Physical (23657  90)  
5 = FinTech (18129  48)  
6 = Infra (15860  37)  
7 = Health (32407  36)  
8 = Biotech (20811  10)  